### Redes de classificação
Selecione um dataset e crie uma rede de classificação. Avalie o resultado com duas configurações diferentes. As configurações devem mudar no mínimo 3 hiperparâmetros (por exemplo, parâmetros do optimizer,  loss, número de camadas, função de ativação, normalização).

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader

In [7]:
weights = MobileNet_V3_Small_Weights.DEFAULT

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=weights.transforms())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=weights.transforms())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Device: {device}")

Device: cuda


In [ ]:
criterion_1 = nn.CrossEntropyLoss()
optimizer_1 = optim.Adam(model.parameters(), lr=0.001, weight_decay=0, betas=(0.9, 0.999))

criterion_2 = nn.MSELoss()
optimizer_2 = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9)

In [11]:
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Zerar os gradientes
            outputs = model(inputs)  # Passar as entradas pela rede
            loss = criterion(outputs, labels)  # Calcular a perda
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos

            # Estatísticas de treinamento
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

In [12]:
def evaluate(model, test_loader, device):
    model.eval()  # Colocar o modelo em modo de avaliação
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
train(model, train_dataloader, criterion_1, optimizer_1, device, num_epochs=15)
evaluate(model, test_dataloader, device)

KeyboardInterrupt: 

In [ ]:
train(model, train_dataloader, criterion_2, optimizer_2, device, num_epochs=10)
evaluate(model, test_dataloader, device)